## Quickstart

### Generate a random sample of orders and store in a sqlite database

Generate a sample of $ 2^{10}$ random orders and store them in a pickle file in `db/stage/example.p`.

In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('../dana/parsers'))
import example_orders

# create the orders
n_orders = 2**10
random_orders = example_orders._generate_orders(n_orders)

# store the orders
path_stage = os.path.relpath('../db/stage')
name_pickle = 'pickle.p'
example_orders._store_orders(random_orders, path_stage, name_pickle)

Read the pickle file in `db/stage/example.p`, create an empty sqlite database in `db/db/example.db` and store the random orders in the database.

In [2]:
sys.path.insert(0, os.path.abspath('../dana/dbadmin'))

import populate_db

# database configurations
db_path = os.path.abspath('../db/db')
db_name = 'example.db'
engine_url = '/'.join(['sqlite:///', db_path, db_name])

# create the folder where the db is stored
if not os.path.exists(db_path):
    os.makedirs(db_path)

# create the engine
engine = populate_db._init_sqlalchemy(engine_url)

# clean the db
# set the flag to True if you want to clean
bool_clean_db = True
if bool_clean_db:
    populate_db._clean_db(engine)

# pickle configurations
path_stage = os.path.abspath('../db/stage')
name_pickle = 'example.p'

# read the orders in the pickle file and store in the db
list_orders = populate_db._read_orders(path_stage, name_pickle)
populate_db._populate_db(engine, list_orders)

Added 1024 records in 0.01830291748046875 secs


Test the main functionalities of the Dana API.
* `get_day` returns all the orders of the input `date` and `symbol`
* `count_orders` returns the number of orders in the database of the input `symbol`

In [4]:
import os

from context import dana

# Create a connection to the api
api = dana.dana.Dana(engine_url) 

# Get the orders executed on AAPL in the julian date 735600
dd = api.get_day(date=735600, symbol='AAPL')

# Print the number of orders and of trades 
print('Number of orders:')
print(len(dd))

# Get the total number of orders in the database executed on AAPL
print('Total number of orders on AAPL:')
print(api.count_orders('AAPL'))

Number of orders:
55
Total number of orders on AAPL:
326
